In [56]:
import pandas as pd

from transformers import T5ForConditionalGeneration, T5Tokenizer

from typing import List

from tqdm.notebook import tqdm

DEVICE = "cuda"

In [51]:
BAD_THEMES = ["Государственная собственность", "Культура", 'МФЦ "Мои документы"', 
              "Памятники и объекты культурного наследия", "Погребение и похоронное дело", 
              "Роспотребнадзор", "Спецпроекты", "Строительство и архитектура", "Торговля",
              "Физическая культура и спорт", "Экология", "Экономика и бизнес", "Электроснабжение"]

In [19]:
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [60]:
def paraphrase(texts: List[str], beams=5, grams=4, do_sample=True):
    x = tokenizer(texts, return_tensors='pt', padding=True).to(model.device)
    
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    
    return tokenizer.batch_decode(out, skip_special_tokens=True)


def generate_new_samples(original_sample: str, n_new_samples: int) -> List[str]:
    return paraphrase([original_sample for _ in range(n_new_samples)])


def generate_new_samples_in_dataset(data: pd.DataFrame, n_new_samples: int = 5, bad_themes: List[str] = BAD_THEMES) -> pd.DataFrame:
    dfs = []
    for _, row in tqdm(data.iterrows(), total=len(data)):
        df = pd.DataFrame(columns=data.columns)
        
        performer, group, text, theme = row["Исполнитель":"Тема"]

        new_samples = generate_new_samples(text, n_new_samples)

        df["Текст инцидента"] = new_samples
        df["Исполнитель"] = performer
        df["Группа тем"] = group
        df["Тема"] = theme

        dfs.append(df)

    return pd.concat(dfs)

In [61]:
data = pd.read_csv("../train_dataset_train_variant2.csv")

bad_data = data.copy()[data["Группа тем"].isin(BAD_THEMES)]
bad_data

,Исполнитель,Группа тем,Текст инцидента,Тема
29,АО ПРО ТКО,Экономика и бизнес,Здравствуйте. Почему Горнозаводское теплоэнер...,Трудоустройство
62,Министерство здравоохранения,Экономика и бизнес,Теперь нам надо будет думать за какие деньги ...,Цены и ценообразование
63,Александровский муниципальный округ Пермского ...,Культура,задал вопрос про статус модерации кинотеатра...,Учреждения культуры
77,Министерство образования,Культура,"Здравствуйте, подскажите, пожалуйста, как опр...",Культурно-досуговые мероприятия
204,Лысьвенский городской округ,Спецпроекты,В какое время 9 мая пойдёт Бессмертный полк?,Спецпроекты
...,...,...,...,...
22352,Лысьвенский городской округ,Культура,"Здравствуйте, когда будут итоги конкурса Лысь...",Учреждения культуры
22354,Город Пермь,Строительство и архитектура,"Здравствуйте, скажите у меня есть земля в нал...",Строительство зданий
22368,Губахинский городской округ,Физическая культура и спорт,А также участие в эстафете принимали студенты...,Спортивные мероприятия
22506,Город Пермь,Культура,"Из Перми пишут здесь 28, 29 и 30 октября разв...",Учреждения культуры


In [62]:
new_data = generate_new_samples_in_dataset(bad_data)

  0%|          | 0/579 [00:00<?, ?it/s]

In [67]:
new_data1 = new_data.drop_duplicates(subset="Текст инцидента")

In [73]:
new_data1.to_csv("generated_train_paraphraser.csv", index=False)